### Imports

In [12]:
import json
import random
import re

from nltk import CFG
from nltk.parse.generate import generate

random.seed(123)

### Items Dictionary from Items Intent
Including
* item name
* slot name
* location details
* neighboring items
* content items (optional)
* adjectives
* hyperonyms

In [29]:
items_dict = {
    "astronautinnen": {
        "slot": "astronautinnen",
        "location": ["links", "oben", "in der ecke"],
        "neighbors": ["ufo", "raumschiff", "anzügen"],
        "content": [],
        "adjectives": ["3", "weiß", "weißen"],
		"hyperonyms": ["menschen", "personen", "frauen"],
    },
    "ballon": {
        "slot": "ballon",
        "location": [
            "rechts",
            "unten",
            "in der ecke",
            "am rand",
            "auf der rechten seite",
            "über",
        ],
        "neighbors": ["boot", "schiff"],
        "content": [],
        "adjectives": ["rot", "rund"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "biene": {
        "slot": "biene",
        "location": ["rechts", "am rand", "auf der rechten seite"],
        "neighbors": [],
        "content": ["streifen"],
        "adjectives": ["gelb", "gestreift"],
		"hyperonyms": ["tier", "insekt", "lebewesen"],
    },
    "boot": {
        "slot": "boot",
        "location": [
            "rechts",
            "unten",
            "in der ecke",
            "am rand",
            "auf der rechten seite",
            "unter",
            "auf",
        ],
        "neighbors": ["ballon", "wasser", "fluss", "roboter"],
        "content": [],
        "adjectives": ["grün", "lächelnd", "schwimmend", "schwimmt"],
		"hyperonyms": ["ding", "gegenstand", "objekt", "fahrzeug"],
    },
    "buchLinksOben": {
        "slot": "buch",
        "location": ["links", "oben", "in der ecke"],
        "neighbors": ["buch", "uhr", "säulen"],
        "content": [],
        "adjectives": ["grün", "blau", "geschlossen"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "buchMitteUnten": {
        "slot": "buch",
        "location": ["in der mitte", "unten", "am rand"],
        "neighbors": ["pfeile", "tisch", "kurven"],
        "content": [],
        "adjectives": ["grün", "geschlossen", "klein"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "buchLinksObenOffen": {
        "slot": "buch",
        "location": ["links", "oben", "in der ecke"],
        "neighbors": ["buch", "uhr", "säulen"],
        "content": [],
        "adjectives": ["aufgeschlagen", "offen", "geöffnet", "klein"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "buchRechtsMitte": {
        "slot": "buch",
        "location": ["rechts", "auf der rechten seite", "unter"],
        "neighbors": ["globus", "spinne", "rauch", "mikroskop"],
        "content": [],
        "adjectives": ["aufgeschlagen", "offen", "geöffnet"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "diagramm": {
        "slot": "diagramm",
        "location": ["rechts", "oben", "am rand", "über", "neben", "an der wand"],
        "neighbors": ["mikroskop", "uhr"],
        "content": ["x", "y", "sigma", "summe", "summenzeichen", "normalverteilung", "gaußkurve", "mathe", "mathematik", "achse"],
        "adjectives": ["bunt", "schwarz", "mathematisch"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "europa": {
        "slot": "europa",
        "location": [
            "unten",
            "links",
            "auf der linken seite",
            "über",
            "unter",
            "neben",
        ],
        "neighbors": ["tisch", "schultasche"],
        "content": ["sterne", "sternen"],
        "adjectives": ["blau", "gelb"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "fff": {
        "slot": "kinder",
        "location": ["oben", "links", "hinter", "unter"],
        "neighbors": ["terminal", "konsole", "astronautinnen", "wolke"],
        "content": ["fahne"],
        "adjectives": ["grün", "2"],
		"hyperonyms": ["menschen", "personen", "kinder"],
    },
    "gleichung": {
        "slot": "gleichung",
        "location": ["unten", "rechts"],
        "neighbors": ["stift", "kolben", "chemie", "labor"],
        "content": ["x", "y", "Gleichheitszeichen"],
        "adjectives": ["weiß"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "globus": {
        "slot": "globus",
        "location": ["in der mitte", "im zentrum"],
        "neighbors": ["wolke", "treppe"],
        "content": ["sockel"],
        "adjectives": ["mittig", "zentral", "rund", "blau", "grün", "rot"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "gluehbirneObenMitteGross": {
        "slot": "glühbirne",
        "location": ["oben", "im oberen teil", "in der mitte", "neben"],
        "neighbors": ["glühbirne", "teleskop"],
        "content": [],
        "adjectives": ["gelb", "groß", "rund"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "gluehbirneObenMitteKlein": {
        "slot": "glühbirne",
        "location": ["oben", "im oberen teil", "in der mitte", "neben"],
        "neighbors": ["glühbirne", "globus"],
        "content": [],
        "adjectives": ["gelb", "klein", "rund", "drei"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "kurven": {
        "slot": "kurven",
        "location": ["unten", "links", "mittig", "links von der mitte", "in der mitte"],
        "neighbors": ["buch", "lineal", "tafel", "wolken"],
        "content": [],
        "adjectives": ["weiß", "bunt", "gelb", "rosa", "pink"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "lab": {
        "slot": "labor",
        "location": ["unten", "rechts", "am rand", "neben"],
        "neighbors": ["wasser", "boot", "ballon"],
        "content": ["terminal", "diy", "kasten"],
        "adjectives": ["eckig", "bunt", "rot"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "mikroskop": {
        "slot": "mikroskop",
        "location": ["oben", "rechts", "neben", "unter"],
        "neighbors": ["diagramm", "buch", "tisch", "pflanze", "kugeln"],
        "content": [],
        "adjectives": ["grau", "grün"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "personLinksMitte": {
        "slot": "person",
        "location": ["links", "in der mitte", "neben"],
        "neighbors": ["taube", "vogel", "wolke", "stift"],
        "content": [],
        "adjectives": ["sitzend", "kauernd", "blond", "lila"],
		"hyperonyms": ["mensch", "person"],
    },
    "personMitteMitte": {
        "slot": "person",
        "location": ["in der mitte", "im zentrum", "neben"],
        "neighbors": ["terminal", "konsole", "ordner", "kästen"],
        "content": ["kittel", "brille", "klemmbrett"],
        "adjectives": ["weiß", "grau"],
		"hyperonyms": ["mensch", "person"],
    },
    "personRechtsMitte": {
        "slot": "person",
        "location": ["in der mitte", "rechts", "neben"],
        "neighbors": ["labor", "magnet", "kolben", "sprechblase"],
        "content": ["kittel", "brille"],
        "adjectives": ["weiß", "blond", "gelb"],
		"hyperonyms": ["mensch", "person"],
    },
    "planet": {
        "slot": "planet",
        "location": [
            "oben",
            "links",
            "auf der linken seite",
            "am Rand",
            "neben",
        ],
        "neighbors": ["buch", "astronautinnen"],
        "content": [],
        "adjectives": ["gelb", "gold", "rund"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "raumschiff": {
        "slot": "raumschiff",
        "location": [
            "oben",
            "links",
            "auf der linken seite",
            "am Rand",
            "neben",
        ],
        "neighbors": ["astronautinnen", "teleskop", "glühbirne", "terminal"],
        "content": [],
        "adjectives": ["rot", "eckig"],
		"hyperonyms": ["ding", "gegenstand", "objekt", "fahrzeug"],
    },
    "spielKonsole": {
        "slot": "spielkonsole",
        "location": ["in der mitte", "im zentrum", "neben"],
        "neighbors": ["glühbirne", "terminal", "astronautinnen", "würfel"],
        "content": ["tasten", "knöpfe"],
        "adjectives": ["eckig", "rot", "schwarz", "grau", "flach"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "stiftLinksMitte": {
        "slot": "stift",
        "location": [
            "mittig",
            "links",
            "auf der linken seite",
            "am Rand",
            "neben",
        ],
        "neighbors": ["person", "würfel", "tafel"],
        "content": [],
        "adjectives": ["grün", "rosa", "weiß", "spitz"],
		"hyperonyms": ["ding", "gegenstand", "objekt", "schreibutensil"],
    },
    "stiftRechtsUnten": {
        "slot": "stift",
        "location": [
            "unten",
            "rechts",
            "auf der rechten seite",
            "am Rand",
            "neben",
        ],
        "neighbors": ["labor", "ballon", "boot", "gleichung", "konsole", "terminal"],
        "content": [],
        "adjectives": ["rot", "spitz", "rosa", "weiß"],
		"hyperonyms": ["ding", "gegenstand", "objekt", "schreibutensil"],
    },
    "tafel": {
        "slot": "tafel",
        "location": [
            "unten",
            "links",
            "auf der linken seite",
            "am Rand",
            "neben",
            "bei",
        ],
        "neighbors": ["würfel", "stift", "buch", "kurven", "europa"],
        "content": ["zwei", "vier", "komma", "punkt"],
        "adjectives": ["grün", "beschriftet", "beschrieben", "weiß"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "teleskop": {
        "slot": "teleskop",
        "location": ["am Rand", "mittig", "neben", "neben"],
        "neighbors": ["glühbirne", "raumschiff"],
        "content": [],
        "adjectives": ["rot", "blau", "rund", "mikroskopähnlich"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "terminalMitteMitte": {
        "slot": "terminal",
        "location": [
            "in der mitte",
            "im zentrum",
            "neben",
        ],
        "neighbors": [
            "globus",
            "spielkonsole",
            "glühbirne",
            "astronautinnen",
            "würfel",  
        ],
        "content": ["schrift", "hello", "world"],
        "adjectives": ["schwarz", "beschriftet", "weiß", "bunt", "grün"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "uhrLinksOben": {
        "slot": "uhr",
        "location": ["oben", "links", "auf der linken seite", "neben"],
        "neighbors": ["buch", "astronautinnen", "person", "planet",],
        "content": ["zeit"],
        "adjectives": ["gelb", "rund"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "uhrRechtsOben": {
        "slot": "uhr",
        "location": [
            "oben",
            "rechts",
            "auf der rechten seite",
            "am Rand",
            "an der Wand",
            "neben",
        ],
        "neighbors": ["diagramm", "mikroskop"],
        "content": ["zeit"],
        "adjectives": ["blau", "rund", "grün"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "wuerfel": {
        "slot": "würfel",
        "location": [
            "mittig",
            "links",
            "auf der linken seite",
            "neben",
        ],
        "neighbors": ["person", "spielkonsole", "kurven", "terminal"],
        "content": ["buchstaben", "a", "b"],
        "adjectives": ["blau", "türkis", "gelb", "lila", "bunt"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
}

In [3]:
model = json.load(open('interaction_model.json', 'r', encoding='utf-8'))

In [4]:
# create a dictionary from interaction model slots {slot: list of slot values and synonyms}
slots = dict()
for s in model["interactionModel"]["languageModel"]["types"]:
    slots[s['name']] = list()
    for v in s['values']:
        if 'synonyms' in v['name']:
            slots[s['name']] += v['name']['synonyms']
        slots[s['name']] += [v['name']['value']]
    # replace ' by `
    for slot, index in zip(slots[s['name']], range(len(slots[s['name']]))):
        slots[s['name']][index] = slot.replace('\'', '`')

In [5]:
slots

{'article': ['wat',
  'was',
  'etwas',
  'dat',
  'des',
  'dor',
  'ditt',
  'dit',
  'diesem',
  'diesen',
  'dieses',
  'diese',
  'dieser',
  'so eine',
  'so einen',
  'so ein',
  'eine',
  'einen',
  'einem',
  'ein',
  'dem',
  'den',
  'das',
  'die',
  'der'],
 'object': ['Absolvent',
  'Student',
  'Gesicht',
  'Smiley',
  'Sterne',
  'Stern',
  'Bildschirm',
  'Podest',
  'Binärcode',
  'Stühle',
  'Stuhl',
  'Seeigel',
  'Mine',
  'Pfeile',
  'Pfeil',
  'Legobaustein',
  'Legostein',
  'Tisch',
  'ABC',
  'Molekül',
  'Atom',
  'Schrank',
  'Delfin',
  'Medaille',
  'Tauben',
  'Taube',
  'Vögel',
  'Vogel',
  'Pfeile',
  'Pfeil',
  'Regenbogen',
  'Reagenzglas',
  'Schultasche',
  'Säulen',
  'Spinne',
  'Wolken',
  'Wolke',
  'Blumentopf',
  'Pflanze',
  'Treppe',
  'Lineal',
  'Büro',
  'Schreibtisch',
  'Giraffenkopf',
  'Giraffe',
  'Zahnrad',
  'other',
  'Sachen',
  'Sache',
  'Teile',
  'Gegenstände',
  'Dinger',
  'Dingens',
  'Dings',
  'Objekt',
  'Gegenstand',


In [7]:
OBJECTS = """"""
for noun in slots['object']:
    OBJECTS += f'N[NUM=sg,CAS=?c,GEN=?g] -> \'{noun}\'\n'

In [9]:
print(OBJECTS)

N[NUM=sg,CAS=?c,GEN=?g] -> 'Absolvent'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Student'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Gesicht'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Smiley'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Sterne'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Stern'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Bildschirm'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Podest'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Binärcode'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Stühle'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Stuhl'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Seeigel'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Mine'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Pfeile'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Pfeil'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Legobaustein'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Legostein'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Tisch'
N[NUM=sg,CAS=?c,GEN=?g] -> 'ABC'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Molekül'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Atom'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Schrank'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Delfin'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Medaille'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Tauben'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Taube'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Vögel'
N[NUM=sg,CAS=?c

In [ ]:
OBJECTS = """
N[NUM=sg,CAS=?c,GEN=m] -> 'Absolvent'
N[NUM=sg,CAS=?c,GEN=m] -> 'Student'
N[NUM=sg,CAS=?c,GEN=n] -> 'Gesicht'
N[NUM=sg,CAS=?c,GEN=m] -> 'Smiley'
N[NUM=pl,CAS=?c,GEN=m] -> 'Sterne'
N[NUM=sg,CAS=?c,GEN=m] -> 'Stern'
N[NUM=sg,CAS=?c,GEN=m] -> 'Bildschirm'
N[NUM=sg,CAS=?c,GEN=n] -> 'Podest'
N[NUM=sg,CAS=?c,GEN=m] -> 'Binärcode'
N[NUM=pl,CAS=?c,GEN=m] -> 'Stühle'
N[NUM=sg,CAS=?c,GEN=m] -> 'Stuhl'
N[NUM=sg,CAS=?c,GEN=m] -> 'Seeigel'
N[NUM=sg,CAS=?c,GEN=f] -> 'Mine'
N[NUM=pl,CAS=?c,GEN=m] -> 'Pfeile'
N[NUM=sg,CAS=?c,GEN=m] -> 'Pfeil'
N[NUM=sg,CAS=?c,GEN=m] -> 'Legobaustein'
N[NUM=sg,CAS=?c,GEN=m] -> 'Legostein'
N[NUM=sg,CAS=?c,GEN=m] -> 'Tisch'
N[NUM=sg,CAS=?c,GEN=n] -> 'ABC'
N[NUM=sg,CAS=?c,GEN=n] -> 'Molekül'
N[NUM=sg,CAS=?c,GEN=n] -> 'Atom'
N[NUM=sg,CAS=?c,GEN=m] -> 'Schrank'
N[NUM=sg,CAS=?c,GEN=m] -> 'Delfin'
N[NUM=sg,CAS=?c,GEN=f] -> 'Medaille'
N[NUM=pl,CAS=?c,GEN=f] -> 'Tauben'
N[NUM=sg,CAS=?c,GEN=f] -> 'Taube'
N[NUM=pl,CAS=?c,GEN=m] -> 'Vögel'
N[NUM=sg,CAS=?c,GEN=m] -> 'Vogel'
N[NUM=pl,CAS=?c,GEN=m] -> 'Pfeile'
N[NUM=sg,CAS=?c,GEN=m] -> 'Pfeil'
N[NUM=sg,CAS=?c,GEN=m] -> 'Regenbogen'
N[NUM=sg,CAS=?c,GEN=n] -> 'Reagenzglas'
N[NUM=sg,CAS=?c,GEN=f] -> 'Schultasche'
N[NUM=pl,CAS=?c,GEN=f] -> 'Säulen'
N[NUM=sg,CAS=?c,GEN=f] -> 'Spinne'
N[NUM=pl,CAS=?c,GEN=f] -> 'Wolken'
N[NUM=sg,CAS=?c,GEN=f] -> 'Wolke'
N[NUM=sg,CAS=?c,GEN=m] -> 'Blumentopf'
N[NUM=sg,CAS=?c,GEN=f] -> 'Pflanze'
N[NUM=sg,CAS=?c,GEN=f] -> 'Treppe'
N[NUM=sg,CAS=?c,GEN=n] -> 'Lineal'
N[NUM=sg,CAS=?c,GEN=n] -> 'Büro'
N[NUM=sg,CAS=?c,GEN=m] -> 'Schreibtisch'
N[NUM=sg,CAS=?c,GEN=m] -> 'Giraffenkopf'
N[NUM=sg,CAS=?c,GEN=f] -> 'Giraffe'
N[NUM=sg,CAS=?c,GEN=n] -> 'Zahnrad'
N[NUM=sg,CAS=?c,GEN=?g] -> 'other'
N[NUM=pl,CAS=?c,GEN=f] -> 'Sachen'
N[NUM=sg,CAS=?c,GEN=f] -> 'Sache'
N[NUM=pl,CAS=?c,GEN=n] -> 'Teile'
N[NUM=pl,CAS=?c,GEN=m] -> 'Gegenstände'
N[NUM=pl,CAS=?c,GEN=n] -> 'Dinger'
N[NUM=sg,CAS=?c,GEN=n] -> 'Dingens'
N[NUM=sg,CAS=?c,GEN=n] -> 'Dings'
N[NUM=sg,CAS=?c,GEN=n] -> 'Objekt'
N[NUM=sg,CAS=?c,GEN=m] -> 'Gegenstand'
N[NUM=sg,CAS=?c,GEN=n] -> 'Teil'
N[NUM=sg,CAS=?c,GEN=n] -> 'Ding'
N[NUM=sg,CAS=?c,GEN=f] -> 'Uhr'
N[NUM=sg,CAS=?c,GEN=m] -> 'Buchstabenwürfel'
N[NUM=sg,CAS=?c,GEN=m] -> 'Würfel'
N[NUM=sg,CAS=?c,GEN=n] -> 'Programmfenster'
N[NUM=sg,CAS=?c,GEN=m] -> 'Bildschirm'
N[NUM=sg,CAS=?c,GEN=m] -> 'Computerbildschirm'
N[NUM=sg,CAS=?c,GEN=f] -> 'Konsole'
N[NUM=sg,CAS=?c,GEN=m] -> 'Terminal'
N[NUM=sg,CAS=?c,GEN=n] -> 'Fernrohr'
N[NUM=sg,CAS=?c,GEN=n] -> 'Teleskop'
N[NUM=sg,CAS=?c,GEN=f] -> 'Schultafel'
N[NUM=sg,CAS=?c,GEN=f] -> 'Tafel'
N[NUM=sg,CAS=?c,GEN=n] -> 'Schreibutensil'
N[NUM=sg,CAS=?c,GEN=m] -> 'Buntstift'
N[NUM=sg,CAS=?c,GEN=m] -> 'Stift'
N[NUM=sg,CAS=?c,GEN=m] -> 'Gameboy'
N[NUM=sg,CAS=?c,GEN=f] -> 'Switch'
N[NUM=sg,CAS=?c,GEN=f] -> 'Spielekonsole'
N[NUM=sg,CAS=?c,GEN=f] -> 'Spielkonsole'
N[NUM=sg,CAS=?c,GEN=n] -> 'Spaceshuttle'
N[NUM=sg,CAS=?c,GEN=n] -> 'Flugzeug'
N[NUM=sg,CAS=?c,GEN=n] -> 'Raumschiff'
N[NUM=sg,CAS=?c,GEN=m] -> 'Saturn'
N[NUM=sg,CAS=?c,GEN=m] -> 'Planet'
N[NUM=sg,CAS=?c,GEN=f] -> 'Ärztin'
N[NUM=sg,CAS=?c,GEN=m] -> 'Arzt'
N[NUM=sg,CAS=?c,GEN=f] -> 'Ingenieurin'
N[NUM=sg,CAS=?c,GEN=m] -> 'Ingenieur'
N[NUM=sg,CAS=?c,GEN=m] -> 'Mann'
N[NUM=sg,CAS=?c,GEN=f] -> 'Frau'
N[NUM=sg,CAS=?c,GEN=f] -> 'Wissenschaftlerin'
N[NUM=sg,CAS=?c,GEN=m] -> 'Wissenschaftler'
N[NUM=sg,CAS=?c,GEN=f] -> 'Forscherin'
N[NUM=sg,CAS=?c,GEN=m] -> 'Forscher'
N[NUM=pl,CAS=?c,GEN=f] -> 'Personen'
N[NUM=sg,CAS=?c,GEN=m] -> 'Mensch'
N[NUM=sg,CAS=?c,GEN=f] -> 'Person'
N[NUM=sg,CAS=?c,GEN=n] -> 'Teleskop'
N[NUM=sg,CAS=?c,GEN=n] -> 'Mikroskop'
N[NUM=sg,CAS=?c,GEN=m] -> 'Kasten'
N[NUM=sg,CAS=?c,GEN=n] -> 'Lab'
N[NUM=sg,CAS=?c,GEN=n] -> 'Labor'
N[NUM=pl,CAS=?c,GEN=f] -> 'Grafiken'
N[NUM=sg,CAS=?c,GEN=n] -> 'Diagramm'
N[NUM=pl,CAS=?c,GEN=f] -> 'Wellen'
N[NUM=pl,CAS=?c,GEN=f] -> 'Kurven'
N[NUM=sg,CAS=?c,GEN=f] -> 'Birne'
N[NUM=sg,CAS=?c,GEN=f] -> 'Glühbirne'
N[NUM=sg,CAS=?c,GEN=m] -> 'Screen'
N[NUM=sg,CAS=?c,GEN=f] -> 'Gleichung'
N[NUM=sg,CAS=?c,GEN=f] -> 'Demo'
N[NUM=pl,CAS=?c,GEN=f] -> 'Klimaaktivistinnen'
N[NUM=pl,CAS=?c,GEN=m] -> 'Klimaaktivisten'
N[NUM=?n,CAS=?c,GEN=?g] -> 'Fridays for Future'
N[NUM=?n,CAS=?c,GEN=?g] -> 'FFF'
N[NUM=pl,CAS=?c,GEN=n] -> 'Kinder mit Flagge'
N[NUM=pl,CAS=?c,GEN=?g] -> 'Friends For Future'
N[NUM=pl,CAS=?c,GEN=?g] -> 'Friends'
N[NUM=sg,CAS=?c,GEN=?g] -> 'Future'
N[NUM=pl,CAS=?c,GEN=n] -> 'Kinder'
N[NUM=sg,CAS=?c,GEN=f] -> 'Flagge'
N[NUM=sg,CAS=?c,GEN=f] -> 'Fahne'
N[NUM=sg,CAS=?c,GEN=f] -> 'EU'
N[NUM=sg,CAS=?c,GEN=f] -> 'Europaflagge'
N[NUM=sg,CAS=?c,GEN=n] -> 'Europa'
N[NUM=sg,CAS=?c,GEN=f] -> 'Tafel'
N[NUM=sg,CAS=?c,GEN=n] -> 'Koordinatensystem'
N[NUM=sg,CAS=?c,GEN=n] -> 'Diagramm'
N[NUM=sg,CAS=?c,GEN=n] -> 'Heft'
N[NUM=sg,CAS=?c,GEN=n] -> 'Buch'
N[NUM=sg,CAS=?c,GEN=n] -> 'Schiff'
N[NUM=sg,CAS=?c,GEN=n] -> 'Boot'
N[NUM=pl,CAS=?c,GEN=m] -> 'Drillinge'
N[NUM=pl,CAS=?c,GEN=f] -> 'Raumfahrerinnen'
N[NUM=pl,CAS=?c,GEN=f] -> 'Frauen'
N[NUM=pl,CAS=?c,GEN=f] -> 'Astronautinnen'
N[NUM=sg,CAS=?c,GEN=m] -> 'Luftballon'
N[NUM=sg,CAS=?c,GEN=m] -> 'Ball'
N[NUM=sg,CAS=?c,GEN=m] -> 'Ballon'
N[NUM=sg,CAS=?c,GEN=n] -> 'Tier'
N[NUM=sg,CAS=?c,GEN=n] -> 'Insekt'
N[NUM=sg,CAS=?c,GEN=f] -> 'Biene'
N[NUM=sg,CAS=?c,GEN=f] -> 'Weltkugel'
N[NUM=sg,CAS=?c,GEN=f] -> 'Erdkugel'
N[NUM=sg,CAS=?c,GEN=m] -> 'Globus'
"""

In [6]:
ARTICLES = """
Det[NUM=sg,CAS=nom,GEN=m] -> 'der'
Det[NUM=sg,CAS=gen,GEN=m] -> 'des'
Det[NUM=sg,CAS=dat,GEN=m] -> 'dem'
Det[NUM=sg,CAS=acc,GEN=m] -> 'den'
Det[NUM=sg,CAS=nom,GEN=f] -> 'die'
Det[NUM=sg,CAS=gen,GEN=f] -> 'der'
Det[NUM=sg,CAS=dat,GEN=f] -> 'der'
Det[NUM=sg,CAS=acc,GEN=f] -> 'die'
Det[NUM=sg,CAS=nom,GEN=n] -> 'das'
Det[NUM=sg,CAS=gen,GEN=n] -> 'des'
Det[NUM=sg,CAS=dat,GEN=n] -> 'dem'
Det[NUM=sg,CAS=acc,GEN=n] -> 'das'
Det[NUM=pl,CAS=nom,GEN=?g] -> 'die'
Det[NUM=pl,CAS=gen,GEN=?g] -> 'der'
Det[NUM=pl,CAS=dat,GEN=?g] -> 'den'
Det[NUM=pl,CAS=acc,GEN=?g] -> 'die'
"""

SyntaxError: invalid syntax (2803656337.py, line 1)

In [10]:
# UTTERANCE STRUCTURE: {beginning} description {neighbors}
# beginning of sentence, aka verbs
BOS = slots['verb'][10:]  # only include parts where NN can follow directly
# item description
descriptions = []
# neighboring items
neighbors = []

In [11]:
BOS

['dann noch',
 'dann',
 'weiter geht`s mit',
 'jetzt',
 'und jetzt',
 'weiter mit',
 'als nächstes',
 'da schwebt',
 'da fliegt',
 'da schwimmt',
 'da ist',
 'da liegt',
 'da steht',
 'siehst du',
 'du siehst',
 'bitte finde',
 'finde',
 'zeich mal',
 'zeich ma',
 'zeich',
 'bitte zeig mir',
 'zeig mir',
 'bitte zeige',
 'zeig mal',
 'zeig',
 'bitte nimm',
 'bitte wähle',
 'wähl mal',
 'wähl',
 'wähle mal',
 'wähle',
 'nimm mal',
 'nimm',
 'ich beschreibe',
 'ich sehe',
 'ich meine',
 'ich wähle',
 'ich sehe',
 'ich denke an',
 'ich nehme',
 'der nächste gegenstand ist',
 'das nächste objekt ist',
 'das ding befindet sich',
 'es befindet sich',
 'das befindet sich',
 'es gibt',
 'das ding ist',
 'der gegenstand ist',
 'das objekt ist',
 'das ist',
 'es ist']

In [ ]:
# FIRST TURN
def first_turn_utterances(i):
    random.choice(i['hyperonym'])
    pass

In [ ]:
# SECOND TURN
def second_turn_utterances(i):
    pass

In [ ]:
# THIRD TURN
def third_turn_utterances(i):
    pass

In [ ]:
for item in items_dict.keys():
    # utterances by turns separated by 2 blank lines
    utterances = first_turn_utterances(item) + ['\n\n'] + second_turn_utterances(item) + ['\n\n'] + third_turn_utterances(item)
    with open(f'utterances/{item}.txt', 'w', encoding='utf-8') as f:
        for u in utterances:
            f.write(u + '\n')